Extracting Population and Land Area Data from CDP Data

In [1]:
#Import required packages 
import pandas as pd
import numpy as np

In [2]:
#Import CDP 2016-2020 Datasets
#Datasets from prior to 2016 have no population or land area data

df2016 = pd.read_csv("https://raw.githubusercontent.com/jadenio/CityEmissionsData/main/CDP%202012-2020/2016_-_Citywide_GHG_Emissions.csv")
df2017 = pd.read_csv("https://raw.githubusercontent.com/jadenio/CityEmissionsData/main/CDP%202012-2020/2017_-_Cities_Community_Wide_Emissions.csv")
df2018 = pd.read_csv("https://raw.githubusercontent.com/jadenio/CityEmissionsData/main/CDP%202012-2020/2018_-_2019_City-wide_Emissions.csv")
df2019 = pd.read_csv("https://raw.githubusercontent.com/jadenio/CityEmissionsData/main/CDP%202012-2020/2019_City-wide_Emissions.csv")
df2020 = pd.read_csv("https://raw.githubusercontent.com/jadenio/CityEmissionsData/main/CDP%202012-2020/2020_-_City-Wide_Emissions.csv")

In [3]:
#Standardize column names 

df2016 = df2016.rename(columns={'Current Population': 'Population'})
df2016 = df2016.rename(columns={'Current Population Year': 'PopulationYear'})
df2016 = df2016.rename(columns={ df2016.columns[23]: "LandArea(sqkm)" })

df2017 = df2017.rename(columns={'Population year': 'PopulationYear'})
df2017 = df2017.rename(columns={'Account number': 'Account Number'})
df2017 = df2017.rename(columns={ df2017.columns[28]: "LandArea(sqkm)" })

df2018 = df2018.rename(columns={'Population Year': 'PopulationYear'})
df2018 = df2018.rename(columns={ df2018.columns[28]: "LandArea(sqkm)" })

df2019 = df2019.rename(columns={'Population Year': 'PopulationYear'})
df2019 = df2019.rename(columns={ df2019.columns[28]: "LandArea(sqkm)" })

df2020 = df2020.rename(columns={'Population Year': 'PopulationYear'})
df2020 = df2020.rename(columns={ df2020.columns[28]: "LandArea(sqkm)" })

Extract Land Area Data to LandArea (la) df

In [4]:
#keep only Account Number and Land Area (year doesn't matter)
la16 = df2016.iloc[:,[0,23]]
la17 = df2017.iloc[:,[0,28]]
la18 = df2018.iloc[:,[1,28]]
la19 = df2019.iloc[:,[1,28]]
la20 = df2020.iloc[:,[1,28]]

In [5]:
#Merge Land Area datasets
frames = [la16, la17, la18, la19, la20]

la = pd.concat(frames)

In [6]:
#Drop duplicate Land Area values for each Account Number
la = la.drop_duplicates(['Account Number'])

In [10]:
#Remove rows with no Land Area information
la = la[la['LandArea(sqkm)'].notna()]

In [9]:
la.to_excel('landarea.xlsx')

Merge Land Area Data to CDPMerge dataset

In [ ]:
cdp = pd.read_excel("CDPMerge.xlsx")

In [118]:
CDPla = cdp.merge(la, on=['Account Number'], how='left')

Extract Population Data to Population (pop) df

In [104]:
#Extract Account Number, Population, Population Year columns
pop16 = df2016.iloc[:,[0,17,16]]
pop17 = df2017.iloc[:,[0,20,21]]
pop18 = df2018.iloc[:,[1,29,30]]
pop19 = df2019.iloc[:,[1,29,30]]
pop20 = df2020.iloc[:,[1,29,30]]

In [105]:
#Merge Population Datasets
frames = [pop16, pop17, pop18, pop19, pop20]

pop = pd.concat(frames)

In [106]:
#Remove duplicates 
pop = pop.drop_duplicates(['Population','Account Number'])

In [107]:
#Remove rows with no Land Area information
pop = pop[pop['Population'].notna()]

In [74]:
#Export final dataset to excel file
pop.to_excel('Population.xlsx')

Take Average Population across all years for each City

In [120]:
avgpop = pop.groupby('Account Number', as_index=False)['Population'].mean()

Merge Population and Land Area data with CDPMerge dataset

In [108]:
#Import CDPMerge dataset
cdp = pd.read_excel("CDPMerge.xlsx")

In [123]:
#Merge Population Data with CDPMerge
cdp_pop = cdp.merge(avgpop, on=['Account Number'], 
    how='left')

In [128]:
#Merge Land Area Data with cdp_pop
cdp_pop_la = cdp_pop.merge(la, on=['Account Number'], 
    how='left')

In [133]:
#Fill missing Population values with column mean
cdp_pop_la["Population"] = cdp_pop_la["Population"].fillna(value=cdp_pop_la["Population"].mean())

In [131]:
#Fill missing Land Area values with column mean
cdp_pop_la["LandArea(sqkm)"] = cdp_pop_la["LandArea(sqkm)"].fillna(value=cdp_pop_la["LandArea(sqkm)"].mean())

In [ ]:
#Export finished dataset to excel file 
cdp_pop_la.to_excel('CDPFinal.xlsx')